In [1]:
import xspec

In [37]:
import os
import numpy as np
import scipy
import scipy.ndimage
import astropy.io.fits as pf
import datetime

In [3]:
os.chdir('/home/qw/astro/soft/nuskybgd')

In [4]:
ls

absrmf.py*  fitab.py*         instrmap.py*      projobs.py*
caldb.py*   getspecnoarf.py*  phafix.py*        __pycache__/
dev/        imrefspec.py*     projinitbgds.py*  README


In [5]:
import caldb

In [6]:
cal = caldb.CalDB(os.environ['CALDB'])

In [29]:
# grade weighting from NGC253 002 obs.
GRADE_WT = [1.00000, 0.124902, 0.117130, 0.114720, 0.118038,
            0.0114296, 0.0101738, 0.0113617, 0.0122017, 0.0157910,
            0.0144079, 0.0145691, 0.0149934, 0.00165462, 0.00194312,
            0.00156128, 0.00143400, 0.00210433, 0.00180735, 0.00140006,
            0.00169704, 0.00189220, 0.00160371, 0.00150188, 0.00168007,
            0.000296983, 0.000364864]

DETNAM = {'DET0': 0, 'DET1': 1, 'DET2': 2, 'DET3': 3}

def shift_image(image, delta):
    return scipy.ndimage.shift(image, delta,
                               mode='wrap', prefilter=False, order=1)

def get_caldb_pixpos(evthdr):
    fpm = evthdr['INSTRUME']
    obsutctime = evthdr['DATE-OBS']
    cal = caldb.CalDB(os.environ['CALDB'])
    pixpospath = cal.getPIXPOS(fpm, 'DET0', obsutctime)

    pixposf = pf.open('%s/%s' % (os.environ['CALDB'], pixpospath))

    pixmap = np.full((360, 360), -1, dtype=np.int32)
    detnum = np.full((360, 360), -1, dtype=np.int32)
    allpdf = np.zeros((360, 360), dtype=np.float64)

    for ext in pixposf:
        if (('EXTNAME' not in ext.header) or
            ('PIXPOS' not in ext.header['EXTNAME']) or
                ('DETNAM' not in ext.header)):
            continue
        idet = int(ext.header['DETNAM'].replace('DET', ''))
        pixpos = ext.data
        for ix in np.arange(32):
            for iy in np.arange(32):
                # Get array indices where all of the following are True
                ii = np.where((pixpos.field('REF_DET1X') != -1) *
                              (pixpos.field('RAWX') == ix) *
                              (pixpos.field('RAWY') == iy) *
                              (pixpos.field('GRADE') <= 26))[0]

                thispdf = np.zeros((360, 360), dtype=np.float64)

                for i in ii:
                    if not np.isnan(pixpos.field('PDF')[i]).any():
                        # No nan value in PDF
                        ref_x = pixpos.field('REF_DET1X')[i]
                        ref_y = pixpos.field('REF_DET1Y')[i]
                        thispdf[ref_y:ref_y + 7, ref_x:ref_x + 7] += (
                            pixpos.field('PDF')[i] *
                            GRADE_WT[pixpos.field('GRADE')[i]])

                ii = np.where(thispdf > allpdf)
                if len(ii) > 0:
                    allpdf[ii] = thispdf[ii]
                    pixmap[ii] = ix + iy * 32
                    detnum[ii] = idet

    pixmap = shift_image(pixmap, [-1, -1])
    detnum = shift_image(detnum, [-1, -1])

    return pixmap, detnum

In [44]:
def get_caldb_instrmap(evthdr):
    """
    Get instrument map image from CALDB, shifted by (-1, -1).

    Returns (image, FITS header).
    """
    fpm = evthdr['INSTRUME']
    obsutctime = evthdr['DATE-OBS']

    cal = caldb.CalDB(os.environ['CALDB'])
    imappath = cal.getINSTRMAP(fpm, obsutctime)
    imapf = pf.open('%s/%s' % (os.environ['CALDB'], imappath))
    try:
        imap = imapf['INSTRMAP'].data
    except KeyError:
        print('Error: INSTRMAP extension not found.')
        return False
    hdr = imapf['INSTRMAP'].header
    hdr['HISTORY'] = nuskybgd_timestamp()
    hdr['HISTORY'] = 'Created inst. map from %s' % os.path.basename(imappath)
    return imap, hdr

def nuskybgd_timestamp():
    return ('%s nuskybgd' %
            datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


In [25]:
def get_badpix_exts(bpixfiles):
    # Collect list of bad pixel extensions, group into FPMA and B
    bpixlist = {'A': [], 'B': []}
    for _ in bpixfiles:
        ff = pf.open(_)
        # Check FPM
        phdr = ff[0].header
        try:
            if phdr['TELESCOP'] != 'NuSTAR':
                print('%s: TELESCOP != NuSTAR, skipping.' % _)
                continue
        except KeyError:
            print('%s: no TELESCOP keyword, skipping.' % _)
            continue

        try:
            if phdr['INSTRUME'] == 'FPMA':
                ab = 'A'
            elif phdr['INSTRUME'] == 'FPMB':
                ab = 'B'
            else:
                print('%s: unknown INSTRUME: %s' % (_, phdr['INSTRUME']))
                continue
        except KeyError:
            print('%s: no INSTRUME keyword, skipping.' % _)
            continue

        for ext in ff[1:]:
            try:
                ehdr = ext.header
                if not (ehdr['XTENSION'] == 'BINTABLE' and
                        'BADPIX' in ehdr['EXTNAME']):
                    continue
                if ehdr['DETNAM'] in DETNAM:
                    bpixlist[ab].append(ext)
                    print('Added %s %s %s to list.' % (_, ab, ehdr['DETNAM']))
                else:
                    print('%s: unknown DETNAM: %s' % (_, ehdr['DETNAM']))
                    continue
            except KeyError:
                # Extension does not have XTENSION, EXTNAME or DETNAM
                continue

    return bpixlist

In [24]:
def apply_badpix(instrmap, bpixexts, pixmap, detnum):
    output = 1. * instrmap

    for ext in bpixexts:
        hh = ext.header
        if ('TSTART' in hh and 'TSTOP' in hh):
            tobs = hh['TSTOP'] - hh['TSTART']
        else:
            tobs = None

        if not ('EXTNAME' in ext.header and
                'BADPIX' in ext.header['EXTNAME'] and
                'DETNAM' in ext.header and
                ext.header['DETNAM'] in DETNAM):
            continue

        idet = DETNAM[ext.header['DETNAM']]
        print(idet)
        badpix = ext.data

        x = badpix.field('RAWX')
        y = badpix.field('RAWY')
        flags = badpix.field('BADFLAG')
        if tobs is not None:
            dt = badpix.field('TIME_STOP') - badpix.field('TIME')

        for i in np.arange(len(x)):
            if ((tobs is not None and dt[i] > 0.8 * tobs) or
                    flags[i][-2] is True):
                ii = np.where((pixmap == x[i] + y[i] * 32) *
                              (detnum == idet))
                output[ii] = 0

            ii = np.where(output > 0)
            output[ii] = detnum[ii] + 1

    return output

In [7]:
os.chdir('/apool/qw/astro/nustar/IC342_X1/90201039002/event_cl')

In [8]:
ls *.evt

nu90201039002A01_cl.evt  nu90201039002A06_cl.evt  nu90201039002B04_cl.evt
nu90201039002A02_cl.evt  nu90201039002B01_cl.evt  nu90201039002B06_cl.evt
nu90201039002A03_cl.evt  nu90201039002B02_cl.evt
nu90201039002A04_cl.evt  nu90201039002B03_cl.evt


In [9]:
evtfh = pf.open('nu90201039002A01_cl.evt')
evthdr = evtfh['EVENTS'].header

In [10]:
cal.getINSTRMAP(evthdr['INSTRUME'], evthdr['DATE-OBS'])

'data/nustar/fpm/bcf/instrmap/nuAinstrmap20100101v004.fits'

In [19]:
pixmap, detnum = get_caldb_pixpos(evthdr)

In [22]:
ds.set_np2arr(pixmap)

1

In [23]:
ds.set_np2arr(detnum)

1

In [30]:
bpixfiles = ['nu90201039002A01_cl.evt']
caldbbpixpath = cal.getBADPIX(
    evthdr['INSTRUME'], 'DET0', evthdr['DATE-OBS'])
bpixfiles.append('%s/%s' % (os.environ['CALDB'], caldbbpixpath))

print('Collecting bad pixel lists...')
bpixexts = get_badpix_exts(bpixfiles)

Added nu90201039002A01_cl.evt A DET0 to list.
Added nu90201039002A01_cl.evt A DET1 to list.
Added nu90201039002A01_cl.evt A DET2 to list.
Added nu90201039002A01_cl.evt A DET3 to list.
Added /soft/astro/heasarc/CALDB/data/nustar/fpm/bcf/badpix/nuAbadpix20100101v002.fits A DET0 to list.
Added /soft/astro/heasarc/CALDB/data/nustar/fpm/bcf/badpix/nuAbadpix20100101v002.fits A DET1 to list.
Added /soft/astro/heasarc/CALDB/data/nustar/fpm/bcf/badpix/nuAbadpix20100101v002.fits A DET2 to list.
Added /soft/astro/heasarc/CALDB/data/nustar/fpm/bcf/badpix/nuAbadpix20100101v002.fits A DET3 to list.


In [45]:
instrmap, header = get_caldb_instrmap(evthdr)

In [46]:
ds.set_np2arr(instrmap)

1

In [47]:
def fpm_parse(keyword):
    if keyword not in ('FPMA', 'FPMB'):
        return False
    else:
        return keyword[-1]

ab = fpm_parse(evthdr['INSTRUME'])
masked_instrmap = apply_badpix(instrmap, bpixexts[ab], pixmap, detnum)

0
1
2
3
0
1
2
3


In [48]:
ds.set_np2arr(masked_instrmap)

1

In [20]:
import pyds9


An instance of ds9 was found to be running before we could
start the 'xpans' name server. You will need to perform a
bit of manual intervention in order to connect this
existing ds9 to Python.

For ds9 version 5.7 and beyond, simply register the
existing ds9 with the xpans name server by selecting the
ds9 File->XPA->Connect menu option. Your ds9 will now be
fully accessible to pyds9 (e.g., it appear in the list
returned by the ds9_targets() routine).

For ds9 versions prior to 5.7, you cannot (easily) register
with xpans, but you can view ds9's File->XPA Information
menu option and pass the value associated with XPA_METHOD
directly to the Python DS9() constructor, e.g.:

    d = DS9('a000101:12345')

The good news is that new instances of ds9 will be
registered with xpans, and will be known to ds9_targets()
and the DS9() constructor.



In [21]:
ds = pyds9.DS9('/tmp/.xpa/DS9_ds9.28641')